In [1]:
import ee  
import geemap
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
output = widgets.Output()

In [2]:
##Google Earth Engine Application  

In [3]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [4]:
Map = geemap.Map(center=[20,78], zoom=4)

In [5]:
# Authenticates and initializes Earth Engine

chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
#region = ee.FeatureCollection("users/hrishippatel/IndiaStates").filterMetadata('NAME_1', 'equals','Uttarakhand')
region=''
#------Dropdown
w = widgets.Dropdown(
    options=['--Select--','Andhra Pradesh','Arunachal Pradesh','Assam','Uttarakhand', 'Haryana', 'Gujarat', 'Uttar Pradesh', 'Punjab'],
    value='--Select--',
    description='Select State:',
)



#onchange Function
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']
        chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
        global region
        region = ee.FeatureCollection("users/hrishippatel/IndiaStates").filterMetadata('NAME_1', 'equals',change['new'])
        precip = chirps.filterDate('2016-01-01', '2019-12-31')
        GujaratPrecip = precip.sum().clip(region).multiply(0.25)
        Map.addLayer(GujaratPrecip, {'min': 0, 'max': 1000, 'palette':"f6ffa9,1fe2b5,1da0fd,2b7bf3,141899"},'Long-term Rainfall')
        year = 2017
        my_fun1(region,year)
        years = ee.List.sequence(1981, 2019)
        yearlyRainfall(year)
        rainfallYears = ee.FeatureCollection(years.map(yearlyRainfall))
        geemap.ee_to_csv(rainfallYears, filename='states1.csv')
        #geemap.ee_export_vector(rainfallYears,region, filename='states1.csv')
        #clear_output(wait=True)
        
       


w.observe(on_change,names='value')
display(w)

def my_fun1(region,year):
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = startDate.advance(1, 'year')
    filtered = chirps \
    .filter(ee.Filter.date(startDate, endDate))

    total = filtered.reduce(ee.Reducer.sum())

    palette = ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494']
    visParams = {
      'min':0,
      'max': 2000,
      'palette': palette
    }
#Map.addLayer(total, visParams, 'Total Precipitation')

# Calculate average rainfall in Bangalore

    stats = total.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = region,
        scale = 5000,
    )
#print(stats.get('precipitation_sum'))
    #print(region)
# Clip the image to the city boundary and display
    totalBangalore = total.clip(region)
    Map.addLayer(totalBangalore, visParams, 'Total Precipitation (states)')

# Calculate average rainfall for past 35+ years

#print(on_change(region))

def yearlyRainfall(year):
        startDate = ee.Date.fromYMD(year, 1, 1)
        endDate = startDate.advance(1, 'year')
        filtered = chirps.filter(ee.Filter.date(startDate, endDate))
        total = filtered.reduce(ee.Reducer.sum())
        stats = total.reduceRegion(
            reducer = ee.Reducer.mean(),
            geometry = region,
            scale = 5000,
            )
        f = ee.Feature(None, {
        'year': year,
        'precipitation': stats.get('precipitation_sum')

        })
        return f
    


# GET DATA
#get_data_button = widgets.Button(
#                    description='Time Series',
#                    button_style='info'
#                    )
#





    
#Export.table.toDrive({
#  'collection': rainfallYears,
#  'folder': 'earthengine',
#  'fileNamePrefix': 'rainfallbyyear',
#  'fileFormat': 'CSV'})
#geemap.ee_export_vector(rainfallYears, filename='sumit.csv')



# DISPLAY BUTTON
#get_data_button.on_click(get_data)
#display(get_data_button)
#get_data_button.observe(get_data)
Map






     

Dropdown(description='Select State:', options=('--Select--', 'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', '…

Map(center=[20, 78], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [7]:
import ipywidgets as widgets
from IPython.display import clear_output
button = widgets.Button(
    description='Display Chart',
    button_style='info'
)
out = widgets.Output()
def on_button_clicked(b):
    button.description = 'Update Time seres'
    with out:
        clear_output(wait=True)
        df = pd.read_csv('states1.csv')
        fig = go.Figure(go.Scatter(x = df['year'], y = df['precipitation'],
        name='Precipitation Time-Series'))
        fig.update_layout(title='Precipitation Time-Series',
        plot_bgcolor='rgb(230, 230,230)',
        showlegend=True)
        #fig.show() 
        display(fig.show())

button.on_click(on_button_clicked)
widgets.VBox([button, out])